In [1]:
# Import Libraries
import os
import pandas as pd
import numpy as np
import joblib
import warnings
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

In [8]:

import lightgbm as lgb
LGBM_AVAILABLE = True


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
LSTM_AVAILABLE = True
    

## Configuration

In [9]:
# Get project directory
NOTEBOOK_DIR = Path.cwd()
PROJECT_DIR = NOTEBOOK_DIR.parent

# Configuration
CONFIG = {
    'train_data_path': PROJECT_DIR / 'data' / 'train.csv',
    'test_data_path': PROJECT_DIR / 'data' / 'test.csv',
    'artifacts_dir': PROJECT_DIR / 'artifacts',
    'results_dir': PROJECT_DIR / 'results',
    'target_column': 'forward_returns',
    'prediction_bounds': (0.0, 2.0),
    'max_volatility_ratio': 1.2,  # 120% of benchmark
    'n_splits': 5,  # For time-series cross-validation
}

# Create results directory if it doesn't exist
CONFIG['results_dir'].mkdir(exist_ok=True)

# Feature list (same as Step 1)
FEATURE_COLS = [
    'V7', 'V13', 'E3', 'M4', 'P11', 'S2', 'I7', 'P2', 'P6', 'E17',
    'M12', 'M11', 'E2', 'I2', 'E9', 'P8', 'P5', 'I5', 'I9', 'V12'
]

## Load and Prepare Data

In [10]:
# Load training data
train_df = pd.read_csv(CONFIG['train_data_path'])
print(f"Loaded training data: {train_df.shape}")
# Safely detect a date column before accessing it to avoid KeyError
date_col = next((c for c in ['date','timestamp','datetime','time','Date','DATE'] if c in train_df.columns), None)
if date_col:
    date_min = pd.to_datetime(train_df[date_col], errors='coerce').min()
    date_max = pd.to_datetime(train_df[date_col], errors='coerce').max()
    if pd.notna(date_min) and pd.notna(date_max):
        print(f"Date range ({date_col}): {date_min} to {date_max}")

Loaded training data: (9021, 98)


In [ ]:
# Data preparation
X = train_df[FEATURE_COLS].copy()
y = train_df[CONFIG['target_column']].copy()
# Safely build a `dates` series from available date-like columns
date_col = next((c for c in ['date','timestamp','datetime','time','Date','DATE'] if c in train_df.columns), None)
if date_col:
    dates = pd.to_datetime(train_df[date_col], errors='coerce')
else:
    dates = pd.Series(pd.NaT, index=train_df.index)

# Handle missing values
X = X.ffill().bfill()

# Remove rows with NaN
valid_idx = ~(X.isnull().any(axis=1) | y.isnull())
X = X[valid_idx]
y = y[valid_idx]
dates = dates[valid_idx]

print(f"Data cleaned: {len(X):,} samples")

Data cleaned: 9,021 samples


## Time-Series Cross-Validation Setup

Using **Walk-Forward** validation to respect temporal ordering.

In [14]:
# Time-series cross-validation
tscv = TimeSeriesSplit(n_splits=CONFIG['n_splits'])

## Model 1: Random Forest Regressor

In [ ]:
print("\n" + "="*70)
print("MODEL 1: RANDOM FOREST")
print("="*70)

rf_results = []

for fold, (train_idx, val_idx) in enumerate(tscv.split(X), 1):
    print(f"\nFold {fold}/{CONFIG['n_splits']}")
    
    # Split data
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_fold)
    X_val_scaled = scaler.transform(X_val_fold)
    
    # Train Random Forest
    rf_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        min_samples_split=20,
        min_samples_leaf=10,
        random_state=42,
        n_jobs=-1
    )
    rf_model.fit(X_train_scaled, y_train_fold)
    
    # Predict
    y_pred = rf_model.predict(X_val_scaled)
    
    # Clip predictions to valid range
    y_pred = np.clip(y_pred, CONFIG['prediction_bounds'][0], CONFIG['prediction_bounds'][1])
    
    # Evaluate
    mse = mean_squared_error(y_val_fold, y_pred)
    mae = mean_absolute_error(y_val_fold, y_pred)
    r2 = r2_score(y_val_fold, y_pred)
    
    rf_results.append({
        'fold': fold,
        'mse': mse,
        'mae': mae,
        'r2': r2
    })
    
    print(f"  MSE: {mse:.6f}")
    print(f"  MAE: {mae:.6f}")
    print(f"  R²: {r2:.6f}")

# Average results
rf_results_df = pd.DataFrame(rf_results)
print(f"\n{'='*50}")
print("RANDOM FOREST - AVERAGE RESULTS")
print(f"{'='*50}")
print(f"MSE: {rf_results_df['mse'].mean():.6f} ± {rf_results_df['mse'].std():.6f}")
print(f"MAE: {rf_results_df['mae'].mean():.6f} ± {rf_results_df['mae'].std():.6f}")
print(f"R²:  {rf_results_df['r2'].mean():.6f} ± {rf_results_df['r2'].std():.6f}")


MODEL 1: RANDOM FOREST

Fold 1/5
  MSE: 0.000148
  MAE: 0.009316
  R²: -0.014001

Fold 2/5
  MSE: 0.000105
  MAE: 0.007374
  R²: -0.080307

Fold 3/5
  MSE: 0.000105
  MAE: 0.007374
  R²: -0.080307

Fold 3/5
  MSE: 0.000177
  MAE: 0.009373
  R²: -0.018052

Fold 4/5
  MSE: 0.000177
  MAE: 0.009373
  R²: -0.018052

Fold 4/5
  MSE: 0.000069
  MAE: 0.005733
  R²: -0.016737

Fold 5/5
  MSE: 0.000069
  MAE: 0.005733
  R²: -0.016737

Fold 5/5


In [ ]:
# Train final Random Forest on all data
print("\nTraining final Random Forest on all data...")

scaler_rf = StandardScaler()
X_scaled = scaler_rf.fit_transform(X)

rf_final = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42,
    n_jobs=-1
)
rf_final.fit(X_scaled, y)

# Save model
joblib.dump(rf_final, CONFIG['artifacts_dir'] / 'rf_model.joblib')
joblib.dump(scaler_rf, CONFIG['artifacts_dir'] / 'rf_scaler.joblib')

print("Random Forest model saved")

## Model 2: LightGBM

In [ ]:
if LGBM_AVAILABLE:
    print("\n" + "="*70)
    print("MODEL 2: LIGHTGBM")
    print("="*70)
    
    lgbm_results = []
    
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X), 1):
        print(f"\nFold {fold}/{CONFIG['n_splits']}")
        
        # Split data
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        
        # Train LightGBM
        lgbm_model = lgb.LGBMRegressor(
            n_estimators=100,
            learning_rate=0.05,
            max_depth=5,
            num_leaves=31,
            min_child_samples=20,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            verbose=-1
        )
        lgbm_model.fit(
            X_train_fold, y_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=False)]
        )
        
        # Predict
        y_pred = lgbm_model.predict(X_val_fold)
        y_pred = np.clip(y_pred, CONFIG['prediction_bounds'][0], CONFIG['prediction_bounds'][1])
        
        # Evaluate
        mse = mean_squared_error(y_val_fold, y_pred)
        mae = mean_absolute_error(y_val_fold, y_pred)
        r2 = r2_score(y_val_fold, y_pred)
        
        lgbm_results.append({
            'fold': fold,
            'mse': mse,
            'mae': mae,
            'r2': r2
        })
        
        print(f"  MSE: {mse:.6f}")
        print(f"  MAE: {mae:.6f}")
        print(f"  R²: {r2:.6f}")
    
    # Average results
    lgbm_results_df = pd.DataFrame(lgbm_results)
    print(f"\n{'='*50}")
    print("LIGHTGBM - AVERAGE RESULTS")
    print(f"{'='*50}")
    print(f"MSE: {lgbm_results_df['mse'].mean():.6f} ± {lgbm_results_df['mse'].std():.6f}")
    print(f"MAE: {lgbm_results_df['mae'].mean():.6f} ± {lgbm_results_df['mae'].std():.6f}")
    print(f"R²:  {lgbm_results_df['r2'].mean():.6f} ± {lgbm_results_df['r2'].std():.6f}")
else:
    print("Skipping LightGBM (not installed)")

In [ ]:
if LGBM_AVAILABLE:
    # Train final LightGBM on all data
    print("\nTraining final LightGBM on all data...")
    
    lgbm_final = lgb.LGBMRegressor(
        n_estimators=100,
        learning_rate=0.05,
        max_depth=5,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        verbose=-1
    )
    lgbm_final.fit(X, y)
    
    # Save model
    joblib.dump(lgbm_final, CONFIG['artifacts_dir'] / 'lgbm_model.joblib')
    
    print("LightGBM model saved")

## Model 3: LSTM (Deep Learning)

In [ ]:
if LSTM_AVAILABLE:
    print("\n" + "="*70)
    print("MODEL 3: LSTM")
    print("="*70)
    
    # Prepare data for LSTM (need 3D input: samples, timesteps, features)
    # We'll use a simple approach: each sample is a single timestep
    
    lstm_results = []
    
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X), 1):
        print(f"\nFold {fold}/{CONFIG['n_splits']}")
        
        # Split data
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        
        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_fold)
        X_val_scaled = scaler.transform(X_val_fold)
        
        # Reshape for LSTM (samples, timesteps, features)
        X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
        X_val_lstm = X_val_scaled.reshape((X_val_scaled.shape[0], 1, X_val_scaled.shape[1]))
        
        # Build LSTM model
        model = Sequential([
            LSTM(50, activation='relu', input_shape=(1, len(FEATURE_COLS)), return_sequences=True),
            Dropout(0.2),
            LSTM(25, activation='relu'),
            Dropout(0.2),
            Dense(1)
        ])
        
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        
        # Early stopping
        early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0)
        
        # Train
        history = model.fit(
            X_train_lstm, y_train_fold,
            validation_data=(X_val_lstm, y_val_fold),
            epochs=100,
            batch_size=32,
            callbacks=[early_stop],
            verbose=0
        )
        
        # Predict
        y_pred = model.predict(X_val_lstm, verbose=0).flatten()
        y_pred = np.clip(y_pred, CONFIG['prediction_bounds'][0], CONFIG['prediction_bounds'][1])
        
        # Evaluate
        mse = mean_squared_error(y_val_fold, y_pred)
        mae = mean_absolute_error(y_val_fold, y_pred)
        r2 = r2_score(y_val_fold, y_pred)
        
        lstm_results.append({
            'fold': fold,
            'mse': mse,
            'mae': mae,
            'r2': r2
        })
        
        print(f"  MSE: {mse:.6f}")
        print(f"  MAE: {mae:.6f}")
        print(f"  R²: {r2:.6f}")
        print(f"  Epochs trained: {len(history.history['loss'])}")
    
    # Average results
    lstm_results_df = pd.DataFrame(lstm_results)
    print(f"\n{'='*50}")
    print("LSTM - AVERAGE RESULTS")
    print(f"{'='*50}")
    print(f"MSE: {lstm_results_df['mse'].mean():.6f} ± {lstm_results_df['mse'].std():.6f}")
    print(f"MAE: {lstm_results_df['mae'].mean():.6f} ± {lstm_results_df['mae'].std():.6f}")
    print(f"R²:  {lstm_results_df['r2'].mean():.6f} ± {lstm_results_df['r2'].std():.6f}")
else:
    print("Skipping LSTM (TensorFlow not installed)")

In [ ]:
if LSTM_AVAILABLE:
    # Train final LSTM on all data
    print("\nTraining final LSTM on all data...")
    
    scaler_lstm = StandardScaler()
    X_scaled = scaler_lstm.fit_transform(X)
    X_lstm = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
    
    lstm_final = Sequential([
        LSTM(50, activation='relu', input_shape=(1, len(FEATURE_COLS)), return_sequences=True),
        Dropout(0.2),
        LSTM(25, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])
    
    lstm_final.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True, verbose=0)
    
    lstm_final.fit(
        X_lstm, y,
        epochs=100,
        batch_size=32,
        callbacks=[early_stop],
        verbose=0
    )
    
    # Save model
    lstm_final.save(CONFIG['artifacts_dir'] / 'lstm_model.h5')
    joblib.dump(scaler_lstm, CONFIG['artifacts_dir'] / 'lstm_scaler.joblib')
    
    print("LSTM model saved")

## Model Comparison

In [ ]:
# Compare all models
print("\n" + "="*70)
print("MODEL COMPARISON SUMMARY")
print("="*70)

comparison = []

# Random Forest
comparison.append({
    'Model': 'Random Forest',
    'MSE': f"{rf_results_df['mse'].mean():.6f} ± {rf_results_df['mse'].std():.6f}",
    'MAE': f"{rf_results_df['mae'].mean():.6f} ± {rf_results_df['mae'].std():.6f}",
    'R²': f"{rf_results_df['r2'].mean():.6f} ± {rf_results_df['r2'].std():.6f}"
})

# LightGBM
if LGBM_AVAILABLE:
    comparison.append({
        'Model': 'LightGBM',
        'MSE': f"{lgbm_results_df['mse'].mean():.6f} ± {lgbm_results_df['mse'].std():.6f}",
        'MAE': f"{lgbm_results_df['mae'].mean():.6f} ± {lgbm_results_df['mae'].std():.6f}",
        'R²': f"{lgbm_results_df['r2'].mean():.6f} ± {lgbm_results_df['r2'].std():.6f}"
    })

# LSTM
if LSTM_AVAILABLE:
    comparison.append({
        'Model': 'LSTM',
        'MSE': f"{lstm_results_df['mse'].mean():.6f} ± {lstm_results_df['mse'].std():.6f}",
        'MAE': f"{lstm_results_df['mae'].mean():.6f} ± {lstm_results_df['mae'].std():.6f}",
        'R²': f"{lstm_results_df['r2'].mean():.6f} ± {lstm_results_df['r2'].std():.6f}"
    })

comparison_df = pd.DataFrame(comparison)
print(comparison_df.to_string(index=False))

# Save comparison
comparison_df.to_csv(CONFIG['results_dir'] / 'model_comparison.csv', index=False)

## Volatility Check

Ensure predictions don't exceed 120% of benchmark volatility.

In [ ]:
# Calculate volatility of predictions vs target
print("\n" + "="*70)
print("VOLATILITY ANALYSIS")
print("="*70)

# Get predictions from best model (Random Forest for now)
X_scaled_all = scaler_rf.transform(X)
predictions = rf_final.predict(X_scaled_all)
predictions = np.clip(predictions, CONFIG['prediction_bounds'][0], CONFIG['prediction_bounds'][1])

# Calculate volatilities
target_vol = y.std()
pred_vol = predictions.std()
vol_ratio = pred_vol / target_vol

print(f"Target (benchmark) volatility: {target_vol:.6f}")
print(f"Prediction volatility: {pred_vol:.6f}")
print(f"Volatility ratio: {vol_ratio:.2%}")
print(f"Max allowed ratio: {CONFIG['max_volatility_ratio']:.0%}")

if vol_ratio <= CONFIG['max_volatility_ratio']:
    print(f"Volatility constraint satisfied")
else:
    print(f"Warning: Volatility exceeds {CONFIG['max_volatility_ratio']:.0%} threshold")

## Feature Importance (Random Forest)

In [ ]:
# Feature importance from Random Forest
feature_importance = pd.DataFrame({
    'feature': FEATURE_COLS,
    'importance': rf_final.feature_importances_
}).sort_values('importance', ascending=False)

print("\n" + "="*70)
print("TOP 10 MOST IMPORTANT FEATURES (Random Forest)")
print("="*70)
print(feature_importance.head(10).to_string(index=False))

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'][:10], feature_importance['importance'][:10])
plt.xlabel('Importance')
plt.title('Top 10 Feature Importance (Random Forest)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig(CONFIG['results_dir'] / 'feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()